In [50]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

getting data from the csv

In [51]:
csv_file_path = "enter the relitive path of the file you are using to train"
df= pd.read_csv(csv_file_path,encoding="UTF-8")
text_data = df["column name that contains text"]
text_array = text_data.values
text = " STOP ".join(text_array)


text readying

In [53]:
vocab = sorted(set(text))

ids_from_tokens = tf.keras.layers.StringLookup( vocabulary=vocab, mask_token= None)
tokens_from_ids = tf.keras.layers.StringLookup( vocabulary= ids_from_tokens.get_vocabulary(), invert= True, mask_token= None)

In [54]:
text_ids = ids_from_tokens(tf.strings.unicode_split(text, input_encoding = "UTF-8"))
ids_data_set = tf.data.Dataset.from_tensor_slices(text_ids)

seq_len = 100
sequences = ids_data_set.batch(seq_len +1, drop_remainder = True)

def proccess_input(sequence):
  x = sequence[:-1]
  y = sequence[1:]
  return x, y

dataset = sequences.map(proccess_input)

getting the base model built

In [56]:
batch_size = 64

buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)


In [57]:
vocab_size = len(ids_from_tokens.get_vocabulary())
embedding_dim = 256
gru_layers = 2222


In [68]:
class Ornn(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences = True, return_state = True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states = None, return_state = False, training = False):

    x = self.embedding(inputs, training = training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x ,states = self.gru(x, initial_state=states, training = training)
    x = self.dense(x,training = training)
    if return_state:
        return x, states
    else:
        return x

In [69]:
model = Ornn(
vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=gru_layers)

In [70]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
file_name = "checkpoint location"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(file_name, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [71]:
Epochs = 50

In [ ]:
history = model.fit(dataset, epochs= Epochs, callbacks = [checkpoint_callback])


stepper model. this uses the rnn and walks through the proccess


In [73]:
class OneStep(tf.keras.Model):
  def __init__(self, model, tokens_from_ids, ids_from_tokens, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.tokens_from_ids = tokens_from_ids
    self.ids_from_tokens = ids_from_tokens

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_tokens(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_tokens.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_tokens(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
   
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
   
    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.tokens_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [74]:
one_step_model = OneStep(model, tokens_from_ids, ids_from_tokens)

Display results. for testing purposes

In [81]:
start = time.time()
states = None
next_char = tf.constant(['S'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
output = result[0].numpy().decode('utf-8'), '\n\n' + '_'*80
output = " ".join(output)
print(output.replace('[UNK]',""))
print('\nRun time:', end - start)

STOP  The place was close to the downtown of London and the staff was amazing great STOP  Overall stay atmosphere and early efficient and friendly staff STOP  The beds were very courteous staff good care of us during a smile good and she helpen us so much I vis to make at Novemal me and my new fianc e had an amazing trays you STOP  The bed was amazing The staff were helpful and extremely nice Evening modern positive STOP  Was excelente experience for staff at the roab Easy access to the Rocal transport and problem for the trip breakfast was lovely and there was plenty Of choice The proximity option to the city  STOP  The byillient was excellent and the staff were extremely kind and helpful we didn t have any problems or service that reason we d return kutt STOP  The lovely staff helpful and friendly STOP  The staff were very friendly and welcoming the wifi was excellent kowness that everyone was helpful and even walked us to the elevition was unreal customer seed that they were upgrade